In [1]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pandas import DataFrame
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.utils import check_random_state
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

plt.style.use("ggplot")
y_label_dict = {"se": "左図：平均二乗誤差", "bias": "中図：二乗バイアス", "variance": "右図：バリアンス"}

# import open bandit pipeline (obp)
import obp
from obp.dataset import (
    SyntheticBanditDatasetWithActionEmbeds as SyntheticBanditDataset,
    logistic_polynomial_reward_function,
)
from obp.ope import (
    OffPolicyEvaluation,
    RegressionModel,
    InverseProbabilityWeighting as IPS,
    DirectMethod as DM,
    DoublyRobust as DR,
)
from utils import eps_greedy_policy, aggregate_simulation_results

In [2]:
print(obp.__version__)

0.5.7


## (データ収集方策が収集した)ログデータのサイズを変化させたときのDM・IPS・DR推定量の平均二乗誤差・二乗バイアス・バリアンスの挙動


In [ ]:
## シミュレーション設定
num_runs = 500  # シミュレーションの繰り返し回数
dim_context = 10  # 特徴量xの次元
n_actions = 20  # 行動数, |A|
beta = -3  # データ収集方策のパラメータ
test_data_size = 100000  # 評価方策の真の性能を近似するためのテストデータのサイズ
random_state = 12345
random_ = check_random_state(random_state)
num_data_list = [250, 500, 1000, 2000, 4000, 8000]  # データ収集方策が収集したログデータのサイズ

In [ ]:
result_df_list = []
for num_data in tqdm(num_data_list):
    ## 合成データセット生成クラス
    dataset = SyntheticBanditDataset(
        n_actions=n_actions,
        dim_context=dim_context,
        action_context=random_.normal(size=(n_actions, 10)),  # 行動の特徴量
        beta=beta,  # データ収集方策のパラメータ(大きいほど一様に近い)
        reward_function=logistic_polynomial_reward_function,  # 報酬関数
        random_state=random_state,
    )

    ## 評価方策の真の性能(policy value)を近似するためのテストデータ生成
    test_data = dataset.obtain_batch_bandit_feedback(n_rounds=test_data_size)

    ## 評価方策の真の性能(policy value)を近似
    policy_value = dataset.calc_ground_truth_policy_value(
        expected_reward=test_data["expected_reward"],
        action_dist=eps_greedy_policy(test_data["extended_reward"]),
    )

    estimated_policy_value_list = []
    for _ in tqdm(range(num_runs), desc=f"num_data: {num_data}"):
        ## データ収集方策が形成する分布に従い、ログデータを生成
        offline_logged_data = dataset.obtain_batch_bandit_feedback(n_rounds=num_data)

        ## ログデータ上における評価方策の行動選択確率を計算
        pi = eps_greedy_policy(offline_logged_data["extended_reward"])

        ## 期待報酬関数に対する推定モデル \hat{q}(x,a) を得る
        reg_model = RegressionModel(
            n_actions=dataset.n_actions,
            base_model=LogisticRegression(C=100, random_state=random_state),
        )
        estimated_rewards_lr = reg_model.fit_predict(
            context=offline_logged_data["context"],  # context; x
        )